In [ ]:
%load_ext autoreload
%autoreload 2
from torchattacks import PGD
from models.discriminator import CLIPSVMDiscriminator
import torchvision
from torchvision import transforms
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

attack = PGD(model, eps=8 / 255, alpha=2 / 255, steps=10, random_start=True)
IMAGE_PATH = "images/dalle/dalle_148.jpg"


# Custom forward method for the attack to handle transformers models
def custom_forward(self, inputs, x):
    with torch.no_grad():
        outputs = model.vision_model(inputs.pixel_values)
        image_features = outputs.last_hidden_state[:, 0, :]
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        return image_features.squeeze().cpu().numpy()


# Assign the custom forward method
attack._targeted = False
attack.forward = custom_forward.__get__(attack)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # This scales pixels to [0, 1]
    ]
)

In [5]:
# Load the image to torch
image = Image.open(IMAGE_PATH)
image = image.convert("RGB")
inputs = processor(images=image, return_tensors="pt")
inputs = inputs.to(model.device)
label = torch.tensor([0]).unsqueeze(0).to(model.device)

In [6]:
# Generate adversarial example
adversarial_image = attack(inputs, torch.tensor([0]))

In [1]:
import lpips

# Define a function to calculate LPIPS difference
def calculate_lpips_difference(image_path1, image_path2, model='alex'):
    # Load the LPIPS model
    loss_fn = lpips.LPIPS(net=model)

    # Load and preprocess the images
    img1 = lpips.im2tensor(lpips.load_image(image_path1))  # Convert to tensor
    img2 = lpips.im2tensor(lpips.load_image(image_path2))  # Convert to tensor

    # Calculate LPIPS difference
    lpips_diff = loss_fn(img1, img2)

    return lpips_diff.item()

/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for fname in os.listdir(source_dir):